## Class-Work 4

Save the state and capital of Nigeria in a DICTIONARY and Create a Quiz in any format.
The quiz should contain different 20 questions for each email
Attach the files and send it to all the emails on the student csv file
The first document would contain the question and the other document would contain the answers.

In [1]:
from bs4 import BeautifulSoup
import requests
import random
import docx
import pandas as pd
import smtplib
import os
from email.message import EmailMessage
import datetime

In [2]:
#Webscrapping for names of states in Nigeria and their capitals 
url = requests.get('https://en.wikipedia.org/wiki/List_of_capitals_of_states_of_Nigeria','lxml')
soup = BeautifulSoup(url.text)
state_list = []
capital_list = []
for x in range(0,72,2):
    state_list.append(soup.findAll('table',class_='wikitable')[0].findAll('td')[x].findAll('a')[0].text)
    #print(state)
for y in range(1,73,2):
    capital_list.append(soup.findAll('table',class_='wikitable')[0].findAll('td')[y].findAll('a')[0].text)
    #print(capital)

In [3]:
state_capital = dict(zip(state_list,capital_list))
student_data = pd.read_csv('Students_contact.csv')

In [4]:
student_data

,NAME,EMAIL,PRESENTER
0,James Ekene,jamesekene8@gmail.com,Yes
1,Olubayode Ebenezer,olubayodeeben@yahoo.com,NaN
2,Nnorom Amaka,helloamaka@gmail.com,Yes
3,Adewumi Mayowa,mayowabammy@gmail.com,NaN
4,Aminat Akinyemi,aaebunoluwa@gmail.com,NaN
5,Aminat Ayoola,TripleAEbun@gmail.com,Yes


In [5]:
username = os.environ.get('email_user')
user_password = os.environ.get('user_pass')
#print(username,user_password)
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
    smtp.login(username, user_password)

In [14]:
for row in student_data.itertuples(index=True, name='Pandas'):
    doc = docx.Document()
    doc2 = docx.Document()
    state = random.sample(list(state_capital.keys()),20)
    #print(state)
    cap_value = dict()
    for each_state in state:
        cap_value.update({each_state:state_capital[each_state]})
    doc.add_paragraph(f'Name: {row.NAME}')
    doc2.add_paragraph('Ansers')
    for index,state_keys in enumerate(cap_value.keys(),1):
        doc.add_paragraph('Question')
        doc.add_paragraph(f'{index}. What is the capital of {state_keys}')
        answer = cap_value[state_keys]
        answer_list = [answer]
        doc2.add_paragraph(f'{index}. {answer}')
        wrong_ans = random.sample(list(state_capital.values()),3)
        answer_update = answer_list + wrong_ans
        while len(set(answer_update)) < 4:
            one_wrong = random.sample(list(state_capital.values()),1)
            if answer in one_wrong:
                continue
            else:
                ans = one_wrong + list(set(answer_update))
                break
        else:
            ans = answer_update
        random.shuffle(ans)
        #print(ans)
        option_list = ['A.)','B.)','C.)','D.)']
        option_update = list(zip(option_list,answer_update))
        #print(option_update)
        for x,y in option_update:
            quest = (x,' ',y)
            doc.add_paragraph(quest)
        doc.add_paragraph('\n')
    doc.save('Questions.docx')
    doc2.save('Answers.docx')
    msg = EmailMessage()
    msg['From'] = username
    msg['To'] = row.EMAIL
    msg['Reply-to'] = username
    msg['Subject'] = 'General Questions'
    msg.set_content(f'Hi {row.NAME},\nPlease find the attached document and answer the following questions')
    with open('Questions.docx','rb') as f:
        content = f.read()
        name = f.name
    msg.add_attachment(content, maintype = 'application', subtype = 'msword', filename = name)
    with open('Answers.docx','rb') as f:
        content = f.read()
        name = f.name
    msg.add_attachment(content, maintype = 'application', subtype = 'msword', filename = name)
    
  
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(username, user_password)
        smtp.send_message(msg)
